Implementation of a ML solution to categories E-commerce products using decision trees.

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from scipy.sparse import vstack

all_df = pd.read_pickle("./at_categories_df.pkl")

In [76]:
class result_filtre:
    def __init__(self,category_value,confused_categories,languages):
        self.category = category(category_value,confused_categories,languages)
        self.includes = includes(self.category)
        self.excludes = excludes(self.includes)
        
    def show(self):
        print(self.category.value)
        print(self.includes.sequence_results.get_best_sequence().sequence, self.includes.sequence_results.get_best_sequence().accuracy)
        print(self.excludes.sequence_results.get_best_sequence().sequence, self.excludes.sequence_results.get_best_sequence().accuracy)

In [77]:
class category:
    def __init__(self,category,confused_categories,languages):
        self.value = category
        self.confused_categories = confused_categories
        self.languages = languages
        self.df = all_df[all_df['product_group'].isin(confused_categories)]
        self.real_classes = self.df['product_group'].values
        self.logical_true_false_sample = self.df['product_group'] == category
        self.total_true = self.df[self.logical_true_false_sample.values].shape[0]

In [78]:
class includes:
    def __init__(self,category):
        self.category = category
        self.logical_true_false_sample = self.category.logical_true_false_sample
        self.vectorizer = vectorize_includes(category)
        self.decision_tree = get_classifier(self.vectorizer.vectorized_data,self.category.real_classes)
        self.sequence_results= sequence_result_include(self)
        if(self.sequence_results.sequences == []):
            self.sequence_results = void_include(self.category.df.shape[0])

        
    def get_report(self):
        return get_report(self.category.real_classes,self.decision_tree.predict(self.vectorizer.vectorized_data))
    
    

In [101]:
class void_include:
    def __init__(self,total_data):
        self.sequences = ["no result"]
        self.existences = [100]
        self.accuracies = [0]
        self.logical_results = [np.repeat(True,total_data)]
    def get_best_sequence(self):
        return best_sequence(self)

In [80]:
class sequence_result_include:
    def __init__(self,includes):
        self.includes = includes
        self.sequences =[]
        self.existences = []
        self.accuracies = []
        self.logical_results = []
        
        node_ = node(includes)
        self.first_includes_object = first_includes(node_)
        
        if(self.first_includes_object.nodes == []):
            self.sequences.append("no result")
            self.existences.append(100)
            self.accuracies.append(0)
            self.logical_results.append(np.repeat(True,self.includes.category.df.shape[0]))
            return None
        
        self.append_sequence([(self.first_includes_object.feature_ids,'include')],self.first_includes_object.feature_names)
        self.second_includes_object = second_includes(self)
        
        
#     def get_sequences(self):
#         self.append_sequence([(self.first_includes_object.feature_ids,'include')])
#         second_includes(self)
        
    
    def append_sequence(self,sequence,feature_names):
        sequence_analysis_ = sequence_analysis(sequence,self.includes)
        if(sequence_analysis_.existence <80):
            return False
        self.sequences.append(list(self.sequences) +[(feature_names,'include')])
        self.existences.append(sequence_analysis_.existence)
        self.accuracies.append(sequence_analysis_.accuracy)
        self.logical_results.append(sequence_analysis_.logical_result)
        
    def get_best_sequence(self):
        return best_sequence(self)

In [81]:
class first_includes:
    

    
    def __init__(self,node):
        self.current_existence = 0
        self.includes = node.includes
        self.current_node = node
        self.nodes = []
        self.feature_ids = []
        self.feature_names = []
        
        self.get_first_includes()
        
        
    def get_first_includes(self):
        if(self.current_node.issplit()):
            self.current_node.divide_samples()
            existence = 100*len(self.current_node.right_true_samples)/self.includes.category.total_true
            if(existence < 20):
                self.current_node.id = self.current_node.includes.decision_tree.tree_.children_left[self.current_node.id]
                self.current_node.samples_stack = self.current_node.left_true_samples + self.current_node.right_true_samples
                return self.get_first_includes() 
            self.current_existence += existence
            if(threshold_reached(self.current_existence)):
                self.nodes.append(self.current_node.id)
                self.feature_ids.append(self.current_node.includes.decision_tree.tree_.feature[self.current_node.id])
                self.feature_names.append(self.current_node.includes.vectorizer.features[self.current_node.includes.decision_tree.tree_.feature[self.current_node.id]])
                return True
            else:
                self.nodes.append(self.current_node.id)
                self.feature_ids.append(self.current_node.includes.decision_tree.tree_.feature[self.current_node.id])
                self.feature_names.append(self.current_node.includes.vectorizer.features[self.current_node.includes.decision_tree.tree_.feature[self.current_node.id]])
                
                self.current_node.id = self.current_node.includes.decision_tree.tree_.children_left[self.current_node.id]
                self.current_node.samples_stack = self.current_node.left_true_samples + self.current_node.right_true_samples
                
                return self.get_first_includes()

        

In [106]:
class second_includes:
    def __init__(self,previous_result):
        self.previous_result = previous_result
        self.previous_includes_object = self.previous_result.first_includes_object
        self.include_attributes = self.previous_includes_object.includes
        self.current_existence = self.previous_includes_object.current_existence
        self.feature_ids = []
        self.feature_names = []
        
        self.get_second_includes()
        
    def get_second_includes(self):
        for node_id in self.previous_includes_object.nodes:
            self.feature_ids.append(self.include_attributes.decision_tree.tree_.feature[self.include_attributes.decision_tree.tree_.children_right[node_id]])
            self.feature_names.append(self.include_attributes.vectorizer.features[self.include_attributes.decision_tree.tree_.feature[self.include_attributes.decision_tree.tree_.children_right[node_id]]])
            self.previous_result.append_sequence([(self.previous_result.first_includes_object.feature_ids,'include')] + [(self.feature_ids,'include')],self.feature_names)

In [107]:
class excludes:
    def __init__(self,previous_includes):
        if(previous_includes.sequence_results.get_best_sequence().accuracy == 100):
            self.sequence_results = void_result()
            return None
        self.category = previous_includes.category
        self.previous_results = previous_includes.sequence_results
        self.logical_result_false_samples = np.logical_and(np.logical_not(previous_includes.category.logical_true_false_sample.values),previous_includes.sequence_results.get_best_sequence().logical_result)
        self.logical_true_false_sample = np.array(previous_includes.category.logical_true_false_sample)[self.previous_results.get_best_sequence().logical_result]
        self.vectorizer = vectorize_excludes(self)
        self.decision_tree = get_classifier(self.vectorizer.vectorized_data,self.category.df[previous_includes.sequence_results.get_best_sequence().logical_result]['product_group'])
        self.sequence_results  = sequence_result_exclude(self)
        if(self.sequence_results.sequences == []):
            self.sequence_results = void_result()
            return None

In [84]:
class void_result:
    
    def get_best_sequence(self):
        return void_excludes()

In [85]:
class void_excludes:
    def __init__(self):
        self.sequence = "no result"
        self.accuracy = "no result"
        self.logical_result = "no result"
        self.existence = "no result"

In [86]:
class sequence_result_exclude:
    def __init__(self,excludes_attributes):
        self.exclude_attributes = excludes_attributes
        self.feature_ids = []
        self.feature_names = []
        self.curent_existence = self.exclude_attributes.previous_results.get_best_sequence().existence
        self.sequences = []
        self.accuracies = []
        self.logical_results = []
        self.existences = []
        
        self.get_sequences()
        
    def get_sequences(self):
        node_ = node(self.exclude_attributes)
        self.get_excludes(node_)
        
        
    def get_excludes(self,node):
        if(node.issplit()):
            sequence_analysis_ = sequence_analysis([(self.feature_ids + [self.exclude_attributes.decision_tree.tree_.feature[node.id]],'exclude')],self.exclude_attributes)
            if(threshold_reached(sequence_analysis_.existence)):
                self.current_existence = sequence_analysis_.existence
                self.feature_ids.append(self.exclude_attributes.decision_tree.tree_.feature[node.id])
                self.feature_names.append(self.exclude_attributes.vectorizer.features[self.exclude_attributes.decision_tree.tree_.feature[node.id]])
                self.sequences.append([(list(self.feature_names),'excludes')])
                self.accuracies.append(sequence_analysis_.accuracy)
                self.logical_results.append(sequence_analysis_.logical_result)
                self.existences.append(self.current_existence)
                node = self.next_node(node)
                return self.get_excludes(node)
            else:
                node = self.next_node(node)
                return self.get_excludes(node)
        else:
            return True
        
    def next_node(self,node):
        new_node = node
        new_node.id = self.exclude_attributes.decision_tree.tree_.children_right[node.id]
        return new_node
    def get_best_sequence(self):
        return best_sequence(self)

In [87]:
class node:
    def __init__(self,includes):
        self.includes = includes
        self.children_left = includes.decision_tree.tree_.children_left
        self.children_right = includes.decision_tree.tree_.children_right
        self.sample_stack =  np.arange(self.includes.vectorizer.vectorized_data.shape[0])
        self.total_true = includes.category.total_true
        self.id = 0
        self.left_true_samples = []
        self.left_false_samples = []
        self.right_true_samples = []
        self.right_false_samples = []
        
    def issplit(self):
        return self.children_left[self.id] != self.children_right[self.id]
    
    def divide_samples(self):
        self.left_true_samples = []
        self.left_false_samples = []
        self.right_true_samples = []
        self.right_false_samples = []
        while len(self.sample_stack)>0:
            sample_id,self.sample_stack = self.sample_stack[-1],self.sample_stack[:-1]
            if(self.includes.vectorizer.vectorized_data[sample_id, self.includes.decision_tree.tree_.feature[self.id]] <0.5 and self.includes.category.logical_true_false_sample.values[sample_id] == True):
                self.left_true_samples.append(sample_id)
            elif(self.includes.vectorizer.vectorized_data[sample_id, self.includes.decision_tree.tree_.feature[self.id]] <0.5 and self.includes.category.logical_true_false_sample.values[sample_id] == False):
                self.left_false_samples.append(sample_id)
            elif(self.includes.vectorizer.vectorized_data[sample_id, self.includes.decision_tree.tree_.feature[self.id]] >0.5 and self.includes.category.logical_true_false_sample.values[sample_id] == True):
                self.right_true_samples.append(sample_id)
            else:
                self.right_false_samples.append(sample_id)

In [88]:
class vectorize_excludes:
    def __init__(self,exclude_object):
        self.vectorized_data,self.features,self.Countvectorizer = self.vectorize(exclude_object)
        
    
    def vectorize(self,exclude_object):
        arr_match_excludes_only = get_name_matches(exclude_object.category.df[exclude_object.logical_result_false_samples]['product_match_vocab'])
        result_arr_excludes_only = concatenize_matches_with_name(exclude_object.category.df[exclude_object.logical_result_false_samples]['display_name'],arr_match_excludes_only)
        arr_match = get_name_matches(exclude_object.category.df[exclude_object.previous_results.get_best_sequence().logical_result]['product_match_vocab'])
        result_arr = concatenize_matches_with_name(exclude_object.category.df[exclude_object.previous_results.get_best_sequence().logical_result]['display_name'],arr_match)
        stop_words = get_stop_words(exclude_object.category.languages)
        return vectorize_data(result_arr_excludes_only,result_arr,stop_words)

In [89]:
def threshold_reached(existence):
    if (existence <80):
        return False
    else:
        return True

In [90]:
class best_sequence:
    def __init__(self,sequence_results):
        best_accuracy_index = np.argmax(sequence_results.accuracies)
        self.sequence = sequence_results.sequences[best_accuracy_index]
        self.accuracy = sequence_results.accuracies[best_accuracy_index]
        self.existence = sequence_results.existences[best_accuracy_index]
        self.logical_result = sequence_results.logical_results[best_accuracy_index]

In [91]:
class sequence_analysis:
    
    def __init__(self,sequence,attributes):
        self.sequence = sequence
        self.logical_result = self.analyse_sequence(self.sequence,attributes.vectorizer.vectorized_data)
        self.total_result,self.total_true = self.get_resulting_count(self.logical_result,attributes.logical_true_false_sample)
        self.accuracy = 100*self.total_true/self.total_result
        self.existence = 100*self.total_true/attributes.category.total_true
        
        
    def analyse_sequence(self,arr_of_tuples_of_ORs,X):
        logicalOR_results= []
        [logicalOR_results.append(self.get_logicalOR_result(tuple_of_ORs,X)) for tuple_of_ORs in arr_of_tuples_of_ORs]
        return np.prod(logicalOR_results,axis=0).astype(bool)
        
    def get_logicalOR_result(self,tuple_of_OR,X):
        if (len(tuple_of_OR[0]) >1):
            logicalOR = np.sum(X[:,tuple_of_OR[0]].reshape((-1,X.shape[0])).toarray().squeeze(),axis=0).astype(bool)
        else:
            logicalOR = X[:,tuple_of_OR[0]].reshape((-1,X.shape[0])).toarray().squeeze().astype(bool)
        if (tuple_of_OR[1] == 'include'):
            return logicalOR
        else:
            return np.logical_not(logicalOR)
        
    def get_resulting_count(self,result_logical_operation,true_or_false_samples):
        logicalAND = np.logical_and(result_logical_operation,true_or_false_samples)
        num_true = 0
        #num_false = 0
        num_total = len([val for val in result_logical_operation if val == True])
        for val in logicalAND:
            if (val ==True):
                num_true +=1
            # else:
            #     num_false +=1
        return num_total,num_true

In [92]:
def get_classifier(X,y_real):
    clf = DecisionTreeClassifier()
    param = get_param_relative_to_data(X,y_real)
    sh = GridSearchCV(clf,param).fit(X,y_real)
    return sh.best_estimator_
   

def get_param_relative_to_data(X,y_real):
    criterion = ['gini','entropy']
    min_samples_leaf = [2,4,6,8,10,15,20]
    min_samples_split = [2,4,8,10,20,30,40]
    max_depth = []
    class_weight = ["balanced"]
    tuning_depth = get_tuning_depth(X,y_real)
    if(tuning_depth >5):
        for max_depth_value in range(int(tuning_depth/5),tuning_depth,int(tuning_depth/5)):
            max_depth.append(max_depth_value)
    else:
        max_depth.append(tuning_depth)
    param = {
        'criterion' : criterion,
        'max_depth' : max_depth,
        'min_samples_leaf' : min_samples_leaf,
        'min_samples_split' : min_samples_split,
        #'class_weight' : class_weight                # remove comment to test
    }
    return param
    
    
def get_tuning_depth(X,y_real):
    not_tuned_clf = DecisionTreeClassifier()
    not_tuned_clf.fit(X,y_real)
    return not_tuned_clf.get_depth()

In [93]:
class vectorize_includes:

    
    
    def __init__(self,category):
        self.vectorized_data,self.features,self.Countvectorizer = self.vectorize(category)
    
    
    
    def vectorize(self,category):
        arr_match_includes_only = get_name_matches(category.df[category.logical_true_false_sample.values]['product_match_vocab'])
        result_arr_includes_only = concatenize_matches_with_name(category.df[category.logical_true_false_sample.values]['display_name'],arr_match_includes_only)
        arr_match = get_name_matches(category.df['product_match_vocab'])
        result_arr = concatenize_matches_with_name(category.df['display_name'],arr_match)
        stop_words = get_stop_words(category.languages)
        return vectorize_data(result_arr_includes_only,result_arr,stop_words)


In [94]:
def get_stop_words(languages):
    stop_words = []
    for language in languages:
        [stop_words.append(word) for word in stopwords.words(language)]
    return stop_words
        
def get_name_matches(match_vocab):
    arr_match = [match.split('____')[0].split('__') for match in match_vocab]
    return arr_match
def concatenize_matches_with_name(display_name,arr_match):
    result_arr = []
    for name, match in zip(display_name,arr_match):
        result_arr.append(name+' '+' '.join(match))
    return result_arr

In [95]:
def vectorize_data(data_to_fit,data_to_transform,stop_words):
    vectorizer = CountVectorizer(binary=True,stop_words=stop_words, token_pattern=r"(?u)\b\w+\b", ngram_range=(1,2), min_df=0.1)
    _ = vectorizer.fit(data_to_fit)
    X = vectorizer.transform(data_to_transform)
    return X, vectorizer.get_feature_names_out(),vectorizer

In [102]:
filtre = result_filtre(455,[458,455,3391,454,459,453,581,457,450,158],['german','english'])

In [103]:
filtre.show()

455
no result 0
[(['gaming'], 'excludes')] 18.498367791077257


In [104]:
#testing on multiple categories
categories = [1401,1402,430,462,920,3109,3391,3951,455,545]
#confused_categories = get_confused_categories(all_df,['german','english'], categories)
confused_categories = [[1401,3123,3124,3122,3136,3126,1416,3400,3117,3114],
                       [1402,1417,3137,3123],
                       [430,562,448,3515,431,3600,437,3955,220,219],
                       [458,462,454,459,453,581,457,450,158],
                       [920,3612,3623,3611,3622,4213,3618,3613,3615,841],
                       [3109,422,3107,385,3108,462,956,3353,3106,165],
                       [3391,458,455,3136,454,453,581,3124,3680,457],
                       [458,3951,3391,454,459,453,581,457,450,158],
                       [458,455,3391,454,459,453,581,457,450,158],
                       [545,3953,3954,3951,512,455,426,3515,3963,3612]]

In [108]:
for category_ , confused_category in zip(categories,confused_categories):
    filtre = result_filtre(category_,confused_category,['german','english'])
    filtre.show()

C:\Users\mikod\anaconda3\envs\categorization\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


1401
[[(['playstation 4'], 'include')], (['ps4'], 'include')] 92.62411347517731
[(['controller'], 'excludes')] 96.17083946980854


C:\Users\mikod\anaconda3\envs\categorization\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


1402
[(['xbox'], 'include')] 76.34660421545668
[(['xbx'], 'excludes')] 95.30791788856305


C:\Users\mikod\anaconda3\envs\categorization\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


430
[(['batterie'], 'include')] 60.8974358974359
[(['knopfzelle'], 'excludes')] 73.07692307692308
462
[(['hdmi'], 'include')] 100.0
no result no result
920
[(['e scooter'], 'include')] 100.0
no result no result


C:\Users\mikod\anaconda3\envs\categorization\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


3109
[(['gaming monitor'], 'include')] 86.86131386861314
[(['75'], 'excludes')] 90.69767441860465
3391
[(['gaming'], 'include')] 64.453125
[(['astro gaming'], 'excludes')] 68.18181818181819
3951
no result 0
[(['gaming'], 'excludes')] 3.851091142490372
455
no result 0
[(['gaming'], 'excludes')] 18.498367791077257
545
no result 0
[(['displayschutzglas'], 'excludes')] 16.946778711484594
